In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
import numpy
import xlrd

In [2]:
def re_cate(label):
    if label > 1.6 :
        return 1
    else:
        return 0

In [3]:
raw_df = pd.read_csv("data_entropy.csv", header = None)
raw_df.rename(columns = {0:'Ae', 1:'Ap', 2:'Rs', 3:'Fz', 4:'label'}, inplace = True)
raw_label = raw_df['label'].values
raw_df['label'] = raw_df.apply(lambda x : re_cate(x.label), axis = 1)
accuracy_label = raw_df['label'].values
raw_df.drop(columns = ['Fz'], axis = 1, inplace = True)
raw_df.drop(columns = ['label'], axis = 1, inplace = True)

In [4]:
raw_df

,Ae,Ap,Rs
0,5.0,2.5,4000
1,5.0,2.5,4500
2,5.0,2.5,5000
3,5.0,2.5,5500
4,5.0,2.5,6000
5,5.0,2.5,6500
6,5.0,2.5,7000
7,5.0,2.5,7500
8,5.0,2.5,8000
9,5.0,2.5,8500


In [5]:
raw_df['ApxRs'] = raw_df['Ap']*raw_df['Rs']
raw_df['AexRs'] = raw_df['Ae']*raw_df['Rs']
raw_df['AexAp'] = raw_df['Ae']*raw_df['Ap']
raw_df['ApxRsxAe'] = raw_df['Ap']*raw_df['Rs']*raw_df['Ae']

In [6]:
raw_feature = raw_df.values
raw_feature_mean = raw_feature.mean(axis = 0)
raw_feature_std = raw_feature.std(axis = 0)
raw_feature_normalized = (raw_feature - raw_feature_mean)/raw_feature_std

In [7]:
grd = GradientBoostingRegressor(n_estimators=100, learning_rate= 0.01, loss= 'ls', max_depth=3)

In [8]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [9]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(raw_feature_normalized, accuracy_label):
    grd.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = grd.predict(raw_feature_normalized[test_index])
    real_label = accuracy_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label > 1.6:
            label = 1
            if label == real_label[real_label_index]:
                pre_label_num += 1
        else:
            label = 0
            if label == real_label[real_label_index]:
                pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 6: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	


In [10]:
grd.fit(raw_feature_normalized, raw_label)
m = grd.apply(raw_feature_normalized)

grd_enc = OneHotEncoder()
grd_enc.fit(m)
n = grd_enc.transform(m).toarray()
X_train_lr=np.append(raw_feature_normalized, n, axis=1)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
m.shape

(380, 100)

In [18]:
n.shape

(380, 800)

In [11]:
lr = LogisticRegression(solver='newton-cg', multi_class='multinomial', C=4, tol=1e-6, max_iter=20)

In [12]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(X_train_lr, accuracy_label):
    lr.fit(X_train_lr[train_index], accuracy_label[train_index])
    pred_smile_label = lr.predict(X_train_lr[test_index])
    real_label = accuracy_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    print(pred_smile_label)
    print(real_label)
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


[0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
[0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
#### In Cross Validation 0: ####
NumofIns Precisely Classified :  34 	 NumofIns :  39 	 Pre_Accuracy :  0.8717948717948718 	
[0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1]
[0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  33 	 NumofIns :  39 	 Pre_Accuracy :  0.8461538461538461 	
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1]
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1
 1]
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  33 	 NumofIns :  38 	 Pre_Accuracy :  0.868421052631579 	


/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1]
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  32 	 NumofIns :  38 	 Pre_Accuracy :  0.8421052631578947 	
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1]
[0 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1]
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  31 	 NumofIns :  38 	 Pre_Accuracy :  0.8157894736842105 	
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1]
[0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1]
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  31 	 NumofIns :  38 	 Pre_Accuracy :  0.8157894736842105 	


/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1
 1]
[0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 1]
#### In Cross Validation 6: ####
NumofIns Precisely Classified :  33 	 NumofIns :  38 	 Pre_Accuracy :  0.868421052631579 	
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 1
 1]
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1
 1]
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  34 	 NumofIns :  38 	 Pre_Accuracy :  0.8947368421052632 	
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
#### In Cross Validation 8: ####
NumofIns Precisely Classified :  32 	 NumofIns :  37 	 Pre_Accuracy :  0.8648648648648649 	
[0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
[1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1]
#### I

/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [13]:
SVM = SVC(kernel='rbf',decision_function_shape='ovo',C=20,shrinking =False,tol =1e-6)

In [14]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(X_train_lr, accuracy_label):
    SVM.fit(X_train_lr[train_index], accuracy_label[train_index])
    pred_smile_label = SVM.predict(X_train_lr[test_index])
    real_label = accuracy_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    print(pred_smile_label)
    print(real_label)
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
[0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
#### In Cross Validation 0: ####
NumofIns Precisely Classified :  34 	 NumofIns :  39 	 Pre_Accuracy :  0.8717948717948718 	
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1]
[0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  33 	 NumofIns :  39 	 Pre_Accuracy :  0.8461538461538461 	
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1]
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1
 1]
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  33 	 NumofIns :  38 	 Pre_Accuracy :  0.868421052631579 	


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1]
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  32 	 NumofIns :  38 	 Pre_Accuracy :  0.8421052631578947 	
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[0 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1]
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1]
[0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1]
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  31 	 NumofIns :  38 	 Pre_Accuracy :  0.8157894736842105 	
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1
 1]
[0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 1]
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1
 1]
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  34 	 NumofIns :  38 	 Pre_Accuracy :  0.8947368421052632 	
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
#### In Cross Validation 8: ####
NumofIns Precisely Classified :  32 	 NumofIns :  37 	 Pre_Accuracy :  0.8648648648648649 	
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1]
#### In Cross Validation 9: ####
NumofIns Precisely Classified :  33 	 NumofIns :  37 	 Pre_Accuracy :  0.8918918918918919 	
mean of NumofIns precisely classified 0.8553652843126527
